In [1]:
# Pkg.add("DistributedArrays")
# Pkg.add("BenchmarkTools")

In [1]:
using BenchmarkTools, DistributedArrays

In [10]:
# println("Using $numprocs processors")

rmprocs(workers())

numprocs = 3
try
    numprocs = parse(Int, ARGS[1])
end

println("Using $numprocs workers")

include(".machines.jl")
add_workers(numprocs)

display(procs())

@everywhere using DistributedArrays
@everywhere using BenchmarkTools

import Base.run

function run(N)
    
    a = rand(N)
    da = distribute(a)
    # da = drand(N)

    #serial_bench = @benchmark sum($a)
    #parallel_bench = @benchmark sum($da)
    
    serial_median   = @elapsed sum(a)
    parallel_median = @elapsed sum(da)

    #serial_median = median(serial_bench.times)
    #parallel_median = median(parallel_bench.times)

    return (serial_median, parallel_median, 
                serial_median / parallel_median)
#    return parallel_median
end

function run_all()
    Ns = Int[]
    all_data = []

    N = 2^20
    
    while N < 10^8
        data = run(N)

        push!(Ns, N)
        push!(all_data, data)

        println("N:", N, "; serial:", data[1],
        "; parallel:", data[2], "; ratio:", data[3])

        N *= 2
    end

    return (Ns, all_data)
end

Ns, all_data = run_all()

Using 3 workers


4-element Array{Int64,1}:
  1
 26
 27
 28

N:1048576; serial:0.000775949; parallel:0.318721877; ratio:0.002434564603169678
N:2097152; serial:0.001372713; parallel:0.001092923; ratio:1.2560015664415518
N:4194304; serial:0.00276338; parallel:0.001640407; ratio:1.6845697439720753
N:8388608; serial:0.0065086; parallel:0.002881631; ratio:2.258651437328374
N:16777216; serial:0.011373443; parallel:0.004955292; ratio:2.295211462815915
N:33554432; serial:0.021115154; parallel:0.009153588; ratio:2.3067625503791516
N:67108864; serial:0.044882957; parallel:0.017691862; ratio:2.536926695449015


([1048576,2097152,4194304,8388608,16777216,33554432,67108864],Any[(0.000775949,0.318722,0.00243456),(0.00137271,0.00109292,1.256),(0.00276338,0.00164041,1.68457),(0.0065086,0.00288163,2.25865),(0.0113734,0.00495529,2.29521),(0.0211152,0.00915359,2.30676),(0.044883,0.0176919,2.53693)])